In [1]:
%matplotlib inline
import glob, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_rows = 200

plt.rcParams['figure.figsize'] = [20, 10]
plt.rc('font', size=20)



# Loading Dataset

## IOT Devices

The dataset has been extracted at `ROOT_PATH`, containing the data for each IOT device. There are 9 devices in total. There are folders for each IOT device containing `benign_traffic.csv` and further two folders for `gafgyt_attacks` and `mirai_attacks`

In [3]:
ROOT_PATH = r'/mnt/data/khiz/dataset/00442'

IOT_DEVS = [ 'Danmini_Doorbell',
             'Ecobee_Thermostat',
             #'Ennio_Doorbell',
             #'B120N10_Baby_Mon',
             '737E_Security_Cam',
             #'838_Security_Cam',
             #'Samsung_Webcam',
             #'1002_Security_Cam',
             #'1003_Security_Cam'
           ]

## Paths to CSVs
We create a python dictionary that contains all the paths to the CSVs for the respective IOT devices

In [4]:
PATHS_DICT = {}

for i in IOT_DEVS:
    PATHS_DICT[i] = {}
    print('[' + i + ']')
    iot_dir = os.path.join(ROOT_PATH, i)
    PATHS_DICT[i]['benign'] = os.path.join(iot_dir, 'benign_traffic.csv')
    print('  ', 'benign_traffic.csv')
    attacks = [ d for d in os.listdir(iot_dir)
                   if os.path.isdir( os.path.join(iot_dir, d)) ]
    for attack in attacks:
        attack_name = attack.split('_')[0]
        
        PATHS_DICT[i][attack_name] = {}
        attack_dir = os.path.join(iot_dir, attack)
        types = [ f for f in os.listdir(attack_dir)
                   if os.path.isfile( os.path.join(attack_dir, f)) ]
        print('  ', attack_name, ': ', types)
        for t in types:
            type_name = t.split('.')[0]
            PATHS_DICT[i][attack_name][type_name] = os.path.join(attack_dir,t)

[Danmini_Doorbell]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']
[Ecobee_Thermostat]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']
[737E_Security_Cam]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']


## Loading CSVs into Pandas
We will create pandas dataframe for each botnet attack on each IOT device. For each datafram we will combine the attack traffic with the benign traffic add the following two columns:
- `traffic_type` : benign or attack (0 or 1)
- `attack_type`  : type of attack e.g, ack, scan etc.

In [5]:
nbaiot_dict = {}
for iot_dev in PATHS_DICT:
    nbaiot_dict[iot_dev] = {}
    b_df = pd.read_csv(PATHS_DICT[iot_dev]['benign'])
    b_df['traffic_type'] = 'benign'
    b_df['attack_type'] = 'benign'
    #nbaiot_dict[iot_dev] = nbaiot_dict[iot_dev].append(df)
    for botnet in [ b for b in PATHS_DICT[iot_dev] if b != 'benign']:
        nbaiot_dict[iot_dev][botnet] = pd.DataFrame()
        nbaiot_dict[iot_dev][botnet] = nbaiot_dict[iot_dev][botnet].append(b_df)
        for attack in PATHS_DICT[iot_dev][botnet]:
            a_df = pd.read_csv(PATHS_DICT[iot_dev][botnet][attack])
            a_df['traffic_type'] = 'attack'
            a_df['attack_type'] = attack
            nbaiot_dict[iot_dev][botnet] = nbaiot_dict[iot_dev][botnet].append(a_df)

In [6]:
for k in nbaiot_dict:
    print(k)
    for b in nbaiot_dict[k]:
        print('  ',b,':','<benign+attack>')

Danmini_Doorbell
   gafgyt : <benign+attack>
   mirai : <benign+attack>
Ecobee_Thermostat
   gafgyt : <benign+attack>
   mirai : <benign+attack>
737E_Security_Cam
   gafgyt : <benign+attack>
   mirai : <benign+attack>


In [ ]:
#nbaiot_dict['Danmini_Doorbell']['mirai']

# Modeling

### Pre processing
Convert the target column `traffic_type` to integer 0 = benigh, 1 = attack 

In [7]:
for dev in nbaiot_dict:
    for botnet in nbaiot_dict[dev]:
        print(dev,botnet)
        nbaiot = nbaiot_dict[dev][botnet]

        nbaiot.loc[ nbaiot['traffic_type']=='attack','traffic_type' ]=1
        nbaiot.loc[ nbaiot['traffic_type']=='benign','traffic_type' ]=0
        nbaiot['traffic_type'] = nbaiot['traffic_type'].astype(int)
        nbaiot = nbaiot.drop(['attack_type'], axis=1)
        
        nbaiot_dict[dev][botnet] = nbaiot

Danmini_Doorbell gafgyt
Danmini_Doorbell mirai
Ecobee_Thermostat gafgyt
Ecobee_Thermostat mirai
737E_Security_Cam gafgyt
737E_Security_Cam mirai


### Classifiers and training functions

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


## Main Loop

In [9]:
classfiers = {
    'KNN' : KNeighborsClassifier(),
    'RFR' : RandomForestClassifier(),
    'DTR' : DecisionTreeClassifier(),
    'ADB' : AdaBoostClassifier(),
    
}

all_results = [
    ['DEVICE','BOTNET', 'CLASSIFIER','TRAIN-SIZE-A', 'TEST-SIZE-B',  'ACCURACY', 'PRECISION', 'RECALL', 'F1-SCORE', 'FALSE-P', 'FALSE-N' ]
]



for dev in nbaiot_dict:
    for botnet in nbaiot_dict[dev]:
        nbaiot = nbaiot_dict[dev][botnet]
        
        print("================", dev, "/", botnet, "================")

        for train_attack_count in [ 
            10, 50, 100, 500, 1000, 5000, 10000, 20000, 40000, 50000 ]:
            
            df_size = len(nbaiot.index)
            df_attack_count = len(nbaiot[ nbaiot['traffic_type'] == 1].index)
            df_benign_count = len(nbaiot[ nbaiot['traffic_type'] == 0].index)
            
            train_benign_count = train_attack_count
            
            df_attack = nbaiot[ nbaiot['traffic_type'] == 1]
            df_train_attack = df_attack.sample(train_attack_count)
            df_test_attack = df_attack.drop(df_train_attack.index)
            
            df_benign = nbaiot[ nbaiot['traffic_type'] == 0]
            df_train_benign = df_benign.sample(train_benign_count)
            df_test_benign = df_benign.drop(df_train_benign.index)
            
            df_train = pd.concat([ df_train_attack, df_train_benign ])
            df_test  = pd.concat([ df_test_attack,  df_test_benign  ]) 
            
            print('-----------------------------------------')            
            print('df_size =',df_size)
            print('df_attack_count =',df_attack_count)
            print('df_benign_count =',df_benign_count)
            print('train_attack_count =',train_attack_count)
            print('train_benign_count =',train_benign_count)
            print('size of df_train =',len(df_train.index))
            print('size of df_test =',len(df_test.index))
            print('-----------------------------------------')
            print('')
            
            X_train = df_train.drop([ "traffic_type" ], axis=1)
            y_train = df_train["traffic_type"]
            
            X_test = df_test.drop([ "traffic_type" ], axis=1)
            y_test = df_test["traffic_type"]
            
            for clf_name in classfiers:
                
                clf = classfiers[clf_name]
                
                clf.fit(X_train, y_train)
                
                y_preds = clf.predict(X_test)

                clrp = classification_report(y_test, y_preds, output_dict=True)
                conf_mat = confusion_matrix(y_test, y_preds)
                fp = conf_mat[0][1]
                fn = conf_mat[1][0]

                res = [
                    dev,
                    botnet,
                    clf_name,
                    
                    train_attack_count,
                    train_benign_count,
                    
                    round(clrp['accuracy'],4),
                    round(clrp['macro avg']['precision'],4),
                    round(clrp['macro avg']['recall'],4),
                    round(clrp['macro avg']['f1-score'],4),
                    fp,
                    fn
                ]
                print(res)
                all_results.append(res)
                print("")


            


================ Danmini_Doorbell / gafgyt ================
-----------------------------------------
df_size = 366198
df_attack_count = 316650
df_benign_count = 49548
train_attack_count = 10
train_benign_count = 10
size of df_train = 20
size of df_test = 366147
-----------------------------------------

['Danmini_Doorbell', 'gafgyt', 'KNN', 10, 10, 0.7933, 0.6929, 0.8661, 0.7117, 1688, 73994]

['Danmini_Doorbell', 'gafgyt', 'RFR', 10, 10, 0.996, 0.9883, 0.9949, 0.9916, 329, 1127]

['Danmini_Doorbell', 'gafgyt', 'DTR', 10, 10, 0.9783, 0.9764, 0.9294, 0.9512, 6818, 1139]

['Danmini_Doorbell', 'gafgyt', 'ADB', 10, 10, 0.9994, 0.9979, 0.9996, 0.9988, 7, 206]

-----------------------------------------
df_size = 366198
df_attack_count = 316650
df_benign_count = 49548
train_attack_count = 50
train_benign_count = 50
size of df_train = 100
size of df_test = 365961
-----------------------------------------

['Danmini_Doorbell', 'gafgyt', 'KNN', 50, 50, 0.9937, 0.9964, 0.9767, 0.9862, 2310, 0]



['Danmini_Doorbell', 'mirai', 'KNN', 10000, 10000, 0.9911, 0.9385, 0.9911, 0.9631, 348, 5468]

['Danmini_Doorbell', 'mirai', 'RFR', 10000, 10000, 0.9999, 0.9996, 1.0, 0.9998, 0, 34]

['Danmini_Doorbell', 'mirai', 'DTR', 10000, 10000, 1.0, 0.9998, 1.0, 0.9999, 2, 15]

['Danmini_Doorbell', 'mirai', 'ADB', 10000, 10000, 1.0, 0.9996, 1.0, 0.9998, 0, 30]

================ Ecobee_Thermostat / gafgyt ================
-----------------------------------------
df_size = 323743
df_attack_count = 310630
df_benign_count = 13113
train_attack_count = 10
train_benign_count = 10
size of df_train = 20
size of df_test = 323697
-----------------------------------------

['Ecobee_Thermostat', 'gafgyt', 'KNN', 10, 10, 0.9605, 0.7522, 0.962, 0.8213, 477, 12324]

['Ecobee_Thermostat', 'gafgyt', 'RFR', 10, 10, 0.9965, 0.9935, 0.9607, 0.9765, 1025, 118]

['Ecobee_Thermostat', 'gafgyt', 'DTR', 10, 10, 0.9985, 0.9992, 0.9809, 0.9899, 500, 0]

['Ecobee_Thermostat', 'gafgyt', 'ADB', 10, 10, 0.9971, 0.994, 0.968, 0

['Ecobee_Thermostat', 'mirai', 'DTR', 5000, 5000, 0.9999, 0.9977, 1.0, 0.9988, 0, 37]

['Ecobee_Thermostat', 'mirai', 'ADB', 5000, 5000, 0.9999, 0.9977, 1.0, 0.9988, 0, 37]

-----------------------------------------
df_size = 525246
df_attack_count = 512133
df_benign_count = 13113
train_attack_count = 10000
train_benign_count = 10000
size of df_train = 20000
size of df_test = 476215
-----------------------------------------

['Ecobee_Thermostat', 'mirai', 'KNN', 10000, 10000, 0.9972, 0.8515, 0.9908, 0.9094, 49, 1294]

['Ecobee_Thermostat', 'mirai', 'RFR', 10000, 10000, 0.9999, 0.9899, 0.9999, 0.9949, 0, 64]

['Ecobee_Thermostat', 'mirai', 'DTR', 10000, 10000, 0.9999, 0.9892, 0.9999, 0.9945, 0, 69]

['Ecobee_Thermostat', 'mirai', 'ADB', 10000, 10000, 0.9999, 0.9899, 0.9999, 0.9949, 0, 64]

================ 737E_Security_Cam / gafgyt ================
-----------------------------------------
df_size = 392250
df_attack_count = 330096
df_benign_count = 62154
train_attack_count = 10
train_b

['737E_Security_Cam', 'mirai', 'ADB', 1000, 1000, 0.9999, 0.9996, 0.9999, 0.9997, 0, 55]

-----------------------------------------
df_size = 498164
df_attack_count = 436010
df_benign_count = 62154
train_attack_count = 5000
train_benign_count = 5000
size of df_train = 10000
size of df_test = 473988
-----------------------------------------

['737E_Security_Cam', 'mirai', 'KNN', 5000, 5000, 0.9907, 0.9701, 0.9874, 0.9785, 976, 3418]

['737E_Security_Cam', 'mirai', 'RFR', 5000, 5000, 0.9999, 0.9995, 0.9999, 0.9997, 0, 55]

['737E_Security_Cam', 'mirai', 'DTR', 5000, 5000, 0.9999, 0.9995, 0.9999, 0.9997, 0, 55]

['737E_Security_Cam', 'mirai', 'ADB', 5000, 5000, 0.9997, 0.9988, 0.9998, 0.9993, 1, 143]

-----------------------------------------
df_size = 498164
df_attack_count = 436010
df_benign_count = 62154
train_attack_count = 10000
train_benign_count = 10000
size of df_train = 20000
size of df_test = 450349
-----------------------------------------

['737E_Security_Cam', 'mirai', 'KNN',

In [10]:
res_df =pd.DataFrame(all_results, index=None)

res_df, res_df.columns = res_df[1:] , res_df.iloc[0]

res_df

,DEVICE,BOTNET,CLASSIFIER,TRAIN-SIZE-A,TEST-SIZE-B,ACCURACY,PRECISION,RECALL,F1-SCORE,FALSE-P,FALSE-N
1,Danmini_Doorbell,gafgyt,KNN,10,10,0.7933,0.6929,0.8661,0.7117,1688,73994
2,Danmini_Doorbell,gafgyt,RFR,10,10,0.996,0.9883,0.9949,0.9916,329,1127
3,Danmini_Doorbell,gafgyt,DTR,10,10,0.9783,0.9764,0.9294,0.9512,6818,1139
4,Danmini_Doorbell,gafgyt,ADB,10,10,0.9994,0.9979,0.9996,0.9988,7,206
5,Danmini_Doorbell,gafgyt,KNN,50,50,0.9937,0.9964,0.9767,0.9862,2310,0
6,Danmini_Doorbell,gafgyt,RFR,50,50,0.9967,0.9899,0.9962,0.993,223,983
7,Danmini_Doorbell,gafgyt,DTR,50,50,0.9902,0.974,0.9848,0.9793,1120,2466
8,Danmini_Doorbell,gafgyt,ADB,50,50,0.9902,0.974,0.9848,0.9793,1120,2466
9,Danmini_Doorbell,gafgyt,KNN,100,100,0.9935,0.9963,0.976,0.9859,2370,0
10,Danmini_Doorbell,gafgyt,RFR,100,100,0.9963,0.9883,0.9961,0.9922,202,1148


In [11]:
# Write to csv
res_csv = res_df.to_csv(index=False)

print(res_csv, file=open('08_results.csv', 'w'))


### Load results from CSV

In [ ]:
res_df = pd.read_csv('02_results.csv')

res_df = res_df[['DEVICE', 'BOTNET', 'CLASSIFIER', 'ACCURACY', 'PRECISION', 'FALSE-P', 'FALSE-N' ]]

res_df

In [ ]:
res_df.set_index(['DEVICE', 'BOTNET', 'CLASSIFIER'], inplace=True)
res_df

In [ ]:
res_df0 = res_df.unstack(level=1)
res_df0

In [ ]:
res_df1 = res_df0.unstack(level=1)
res_df1

In [ ]:
res_df1.plot(kind='bar')